In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split

# Constants
num_samples = 10000
train_size = int(0.8 * num_samples)
val_size = num_samples - train_size
num_epochs = 1000

# Dataset
x_values = torch.rand(num_samples, 1) * 10 + 20 # Random values between 0 and 10
y_values = torch.rand(num_samples, 1) * 10 + 10  # To ensure no division by zero

inputs = torch.cat((x_values, y_values), dim=1)
outputs = x_values / y_values

train_dataset, val_dataset = random_split(TensorDataset(inputs, outputs), [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Definition
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim, depth):
        super(MLP, self).__init__()
        layers = [nn.Linear(input_dim, hidden_dim), nn.ReLU()]
        for _ in range(depth - 2):
            layers.extend([nn.Linear(hidden_dim, hidden_dim), nn.ReLU()])
        layers.append(nn.Linear(hidden_dim, output_dim))
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

model = MLP(input_dim=2, output_dim=1, hidden_dim=32, depth=4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Training setup
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    # train_losses.append(train_loss / len(train_loader))
    
    if epoch % 100 == 0:
        # Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()
        # val_losses.append(val_loss / len(val_loader))
        print(f"Epoch {epoch}:\t", train_loss / len(train_loader), val_loss / len(val_loader))
# train_losses, val_losses


Epoch 0:	 0.05578207740560174 0.010829023048577328


KeyboardInterrupt: 